In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/malaya/master/session/constituency/src/trees_newline.py
# !wget https://github.com/aisingapore/seacorenlp-data/raw/main/id/constituency/train.txt
# !wget https://github.com/aisingapore/seacorenlp-data/raw/main/id/constituency/dev.txt
# !wget https://github.com/aisingapore/seacorenlp-data/raw/main/id/constituency/test.txt

In [6]:
BERT_TOKEN_MAPPING = {
    "-LRB-": "(",
    "-RRB-": ")",
    "-LCB-": "{",
    "-RCB-": "}",
    "-LSB-": "[",
    "-RSB-": "]",
    "``": '"',
    "''": '"',
    "`": "'",
    '«': '"',
    '»': '"',
    '‘': "'",
    '’': "'",
    '“': '"',
    '”': '"',
    '„': '"',
    '‹': "'",
    '›': "'",
    "\u2013": "--", # en dash
    "\u2014": "--", # em dash
    }

def process_word(word):
    word = word.replace('\\/', '/').replace('\\*', '*')
    # Mid-token punctuation occurs in biomedical text
    word = word.replace('-LSB-', '[').replace('-RSB-', ']')
    word = word.replace('-LRB-', '(').replace('-RRB-', ')')
    if word == "n't" and cleaned_words:
        cleaned_words[-1] = cleaned_words[-1] + "n"
        word = "'t"
    return word

In [7]:
import trees_newline as trees

train_treebank = trees.load_trees('train.txt')
train_parse = [tree.convert() for tree in train_treebank]

In [8]:
tree = train_parse[0]
[process_word(leaf.word) for leaf in tree.leaves()]

['Setelah',
 'uji',
 'coba',
 'itu',
 ',',
 'Ketua',
 'Umum',
 'PSSI',
 'Nurdin',
 'Halid',
 'meminta',
 'Badan',
 'Tim',
 'Nasional',
 'mengevaluasi',
 'Riedl',
 'karena',
 'dinilai',
 'tak',
 'memberi',
 'kemajuan',
 'nyata',
 'bagi',
 'tim',
 '.']

In [12]:
from glob import glob

punct = ',.?!:;'

prepare = []

for f in glob('*.txt'):
    with open(f) as fopen:
        train = list(filter(None, fopen.read().split('\n')))


    for t in train:
        try:
            tree = trees.tree_from_str(t).convert()
            words = ' '.join([process_word(leaf.word) for leaf in tree.leaves()])
            for p in punct:
                words = words.replace(f' {p} ', f'{p} ')
            prepare.append((words, t))
        except Exception as e:
            pass

In [14]:
prepare[-1]

('Di pertengahan perjalanan mereka digantikan oleh kelompok Kapten Widyo Kirono Kusumowidjojo, yang menerbangkan pesawat sampai Amsterdam .',
 '(S (PP (PPO Di) (NP (NNO pertengahan) (NNO perjalanan))) (NP (PRN mereka)) (VP (VBP digantikan) (PP (PPO oleh) (NP (NP (NNO kelompok) (NNO Kapten) (NNP Widyo) (NNP Kirono) (NNP Kusumowidjojo)) (PUN ,) (RPN (PRR yang) (VP (VBT menerbangkan) (NP (NNO pesawat)) (PP (PPO sampai) (NNP Amsterdam))))))) (PUN .))')

In [15]:
import json

with open('constituency.jsonl', 'w') as fopen:
    for t in prepare:
        
        data = {
            'prompt_input': None,
            'input': 'constituency parsing: ' + t[0],
            'output': t[1],
        }
        fopen.write(f'{json.dumps(data)}\n')